In [1]:
%pip install opencv-python matplotlib

Note: you may need to restart the kernel to use updated packages.


In [ ]:
import platform
platform.architecture()
import tensorflow as tf

In [ ]:

gpu_available = tf.test.is_gpu_available()
is_cuda_gpu_available = tf.test.is_gpu_available(cuda_only=True)
is_cuda_gpu_min_3 = tf.test.is_gpu_available(True, (3,0))
print(tf.config.list_physical_devices('GPU'))
print(gpu_available)
print(is_cuda_gpu_available)
print(is_cuda_gpu_min_3)